# Titanic Survival Prediction

In [1]:
train_location = 'train.csv'
test_location = 'test.csv'
Original_test_location = 'testOriginal.csv'

In [2]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
import matplotlib as plt

C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
trainData = pd.read_csv(train_location)
# dropping any row where age is nan because it is a huge factor
trainData = trainData.dropna(how='any',axis=0) 
X_train = trainData.iloc[:,:7].values
Y_train = trainData.iloc[:,7].values

In [4]:
testData = pd.read_csv(test_location)
testOriginalData = pd.read_csv(Original_test_location)
X_test = testData.iloc[:,:7].values

In [5]:
trainData.shape

(712, 8)

In [6]:
testData.shape

(418, 7)

In [7]:
trainData;

In [8]:
# dont drop test values as it is required to submit 418 rows of predictions
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)
testImputer = imputer.fit(X_test[:,:])
X_test[:,:] = testImputer.transform(X_test[:,:])

In [9]:
X_train

array([[ 3.    ,  0.    , 22.    , ...,  0.    ,  7.25  ,  1.    ],
       [ 1.    ,  1.    , 38.    , ...,  0.    , 71.2833,  0.    ],
       [ 3.    ,  1.    , 26.    , ...,  0.    ,  7.925 ,  1.    ],
       ...,
       [ 1.    ,  1.    , 19.    , ...,  0.    , 30.    ,  1.    ],
       [ 1.    ,  0.    , 26.    , ...,  0.    , 30.    ,  0.    ],
       [ 3.    ,  0.    , 32.    , ...,  0.    ,  7.75  ,  2.    ]])

In [10]:
# scaling 
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [11]:
X_train.shape

(712, 7)

In [12]:
Y_train.shape

(712,)

In [13]:
def float_range(start,stop,step):
    l = []
    l.append(start);
    
    while(start+step<=stop):
        start = start + step
        l.append(start)
        
    return l;

In [14]:
import numpy as np

In [15]:
np.linspace(0.01,0.3,8)

array([0.01      , 0.05142857, 0.09285714, 0.13428571, 0.17571429,
       0.21714286, 0.25857143, 0.3       ])

In [16]:
l = np.floor(np.linspace(0,1000,30))

In [17]:
l

array([   0.,   34.,   68.,  103.,  137.,  172.,  206.,  241.,  275.,
        310.,  344.,  379.,  413.,  448.,  482.,  517.,  551.,  586.,
        620.,  655.,  689.,  724.,  758.,  793.,  827.,  862.,  896.,
        931.,  965., 1000.])

In [18]:
range(1,6,2)

range(1, 6, 2)

# XGBoost

In [19]:
XGB_param_grid = {
                    'max_depth':[3,4,5,6,7,8,9,10],
                    'min_child_weight':[1,3,5,7,9],
                    'learning_rate':[0.001,0.01,0.05,0.1,0.15,0.21,0.25],
                    'n_estimators':[10,30,60,100,200,300,500,700,1000],
                    'gamma':[i/10.0 for i in range(0,5)],
                    'subsample':[i/100.0 for i in range(75,90,5)],
                    'colsample_bytree':[i/100.0 for i in range(75,90,5)],
                    'reg_alpha':[1e-5, 1e-2, 0, 0.001, 0.005, 0.05, 0.1, 1, 100 ]                   
                 }

### Init signature:  

XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=100, silent=True, objective='binary:logistic', booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None, **kwargs)

In [20]:
from xgboost.sklearn import XGBClassifier

In [21]:
from sklearn.grid_search import RandomizedSearchCV

In [22]:
xgb = XGBClassifier()

In [23]:
Y_train.shape

(712,)

In [24]:
%%time 
randcvXGB = RandomizedSearchCV(xgb, XGB_param_grid, cv=5, n_iter=100, scoring='accuracy',verbose=2)
randcvXGB.fit(X_train, Y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] subsample=0.75, reg_alpha=0.01, n_estimators=60, min_child_weight=3, max_depth=9, learning_rate=0.01, gamma=0.0, colsample_bytree=0.85 
[CV]  subsample=0.75, reg_alpha=0.01, n_estimators=60, min_child_weight=3, max_depth=9, learning_rate=0.01, gamma=0.0, colsample_bytree=0.85 -   0.1s
[CV] subsample=0.75, reg_alpha=0.01, n_estimators=60, min_child_weight=3, max_depth=9, learning_rate=0.01, gamma=0.0, colsample_bytree=0.85 
[CV]  subsample=0.75, reg_alpha=0.01, n_estimators=60, min_child_weight=3, max_depth=9, learning_rate=0.01, gamma=0.0, colsample_bytree=0.85 -   0.0s
[CV] subsample=0.75, reg_alpha=0.01, n_estimators=60, min_child_weight=3, max_depth=9, learning_rate=0.01, gamma=0.0, colsample_bytree=0.85 

C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: De


[CV]  subsample=0.75, reg_alpha=0.01, n_estimators=60, min_child_weight=3, max_depth=9, learning_rate=0.01, gamma=0.0, colsample_bytree=0.85 -   0.0s
[CV] subsample=0.75, reg_alpha=0.01, n_estimators=60, min_child_weight=3, max_depth=9, learning_rate=0.01, gamma=0.0, colsample_bytree=0.85 
[CV]  subsample=0.75, reg_alpha=0.01, n_estimators=60, min_child_weight=3, max_depth=9, learning_rate=0.01, gamma=0.0, colsample_bytree=0.85 -   0.0s
[CV] subsample=0.75, reg_alpha=0.01, n_estimators=60, min_child_weight=3, max_depth=9, learning_rate=0.01, gamma=0.0, colsample_bytree=0.85 
[CV]  subsample=0.75, reg_alpha=0.01, n_estimators=60, min_child_weight=3, max_depth=9, learning_rate=0.01, gamma=0.0, colsample_bytree=0.85 -   0.0s
[CV] subsample=0.85, reg_alpha=0.01, n_estimators=700, min_child_weight=5, max_depth=10, learning_rate=0.21, gamma=0.1, colsample_bytree=0.75 


C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  subsample=0.85, reg_alpha=0.01, n_estimators=700, min_child_weight=5, max_depth=10, learning_rate=0.21, gamma=0.1, colsample_bytree=0.75 -   0.2s
[CV] subsample=0.85, reg_alpha=0.01, n_estimators=700, min_child_weight=5, max_depth=10, learning_rate=0.21, gamma=0.1, colsample_bytree=0.75 


...

[CV]  subsample=0.85, reg_alpha=0.005, n_estimators=700, min_child_weight=5, max_depth=9, learning_rate=0.01, gamma=0.3, colsample_bytree=0.8 -   0.2s
[CV] subsample=0.85, reg_alpha=0.005, n_estimators=700, min_child_weight=5, max_depth=9, learning_rate=0.01, gamma=0.3, colsample_bytree=0.8 


C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  subsample=0.85, reg_alpha=0.005, n_estimators=700, min_child_weight=5, max_depth=9, learning_rate=0.01, gamma=0.3, colsample_bytree=0.8 -   0.2s
[CV] subsample=0.85, reg_alpha=0.005, n_estimators=700, min_child_weight=5, max_depth=9, learning_rate=0.01, gamma=0.3, colsample_bytree=0.8 


C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.0min finished


[CV]  subsample=0.85, reg_alpha=0.005, n_estimators=700, min_child_weight=5, max_depth=9, learning_rate=0.01, gamma=0.3, colsample_bytree=0.8 -   0.2s
Wall time: 1min 2s


In [25]:
print(randcvXGB.best_score_)
print(randcvXGB.best_params_)

0.8356741573033708
{'subsample': 0.75, 'reg_alpha': 0.05, 'n_estimators': 700, 'min_child_weight': 5, 'max_depth': 3, 'learning_rate': 0.15, 'gamma': 0.4, 'colsample_bytree': 0.8}


In [35]:
tunedXGB= XGBClassifier(subsample=0.75, reg_alpha= 0.05 , min_child_weight= 5, learning_rate= 0.15, gamma= 0.4, colsample_bytree= 0.8, n_estimators=700, max_depth=3)

In [36]:
tunedXGB.fit(X_train,Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.4, learning_rate=0.15,
       max_delta_step=0, max_depth=3, min_child_weight=5, missing=None,
       n_estimators=700, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0.05,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.75)

In [37]:
predictions = tunedXGB.predict(X_test)

C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [38]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [39]:
X_test.shape

(418, 7)

In [40]:
testOriginalData.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [43]:
results = pd.DataFrame({
    "PassengerId": testOriginalData['PassengerId'],
    "Survived": predictions})

In [45]:
results.to_csv('titanic_XGB2.csv', index=False)   